In [2]:
import pandas as pd
import re

from tqdm.auto import tqdm
tqdm.pandas()

/share/u/arunas/miniconda3/envs/broca/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Genders for subjects based on italian

In [166]:
genderDict = {
    "pear": "kar",
    "author": "kon",
    "authors": "kons",
    "banana": "kar",
    "biscuit": "kon",
    "book": "kon",
    "bottle": "kar",
    "box": "kar",
    "boy": "kon",
    "boys": "kons",
    "bulb": "kar",
    "cabinet": "kar",
    "cap": "kon",
    "cat": "kon",
    "cats": "kons",
    "chapter": "kon",
    "chalk": "kon",
    "cup": "kar",
    "cucumber": "kon",
    "dog": "kon",
    "dogs": "kons",
    "fish": "kon",
    "fruit": "kar",
    "girl": "kar",
    "girls": "kars",
    "hill": "kar",
    "man": "kon",
    "men": "kons",
    "meal": "kon",
    "mountain": "kar",
    "mouse": "kon",
    "newspaper": "kon",
    "pizza": "kar",
    "poet": "kon",
    "poets": "kons",
    "poem": "kar",
    "rock": "kon",
    "roof": "kon",
    "speaker": "kon",
    "speakers": "kons",
    "staircase": "kar",
    "story": "kar",
    "teacher": "kon",
    "teachers": "kons",
    "toy": "kon",
    "tree": "kar",
    "woman": "kar",
    "women": "kars",
    "writer": "kon",
    "writers": "kons"
}

pastTense = {
    'climbs' : 'climbed',
    'reads': 'read',
    'carries': 'carried',
    'eats': 'ate',
    'holds': 'held',
    'takes' :'took',
    'brings': 'brought',
    'reads': 'read',
    'climb' : 'climbed',
    'read': 'read',
    'carry': 'carried',
    'eat': 'ate',
    'hold': 'held',
    'take' :'took',
    'bring': 'brought',
    'read': 'read'
}

infinitive = {
    'climbs' : 'to climb',
    'reads': 'to read',
    'carries': 'to carry',
    'eats': 'to eat',
    'holds': 'to hold',
    'takes' : 'to take',
    'brings': 'to bring',
    'reads': 'to read',
    'climb' : 'to climb',
    'read': 'to read',
    'carry': 'to carry',
    'eat': 'to eat',
    'hold': 'to hold',
    'take' : 'to take',
    'bring': 'to bring',
    'read': 'to read'
}

pluralObjects = {
    'fish': 'fish',
    'mouse': 'mice',
    'bottle': 'bottles',
    'newspaper': 'newspapers',
    'chalk': 'chalks',
    'box': 'boxes',
    'cap': 'caps',
    'bulb': 'bulbs',
    'cup': 'cups',
    'toy': 'toys',
    'staircase': 'staircases',
    'rock': 'rocks',
    'hill': 'hills',
    'mountain': 'mountains',
    'roof': 'roofs',
    'tree': 'trees',
    'biscuit': 'biscuits',
    'banana': 'bananas',
    'pear': 'pears',
    'meal': 'meals',
    'fruit': 'fruits',
    'cucumber': 'cucumbers',
    'pizza': 'pizzas',
    'book': 'books',
    'poem': 'poems',
    'story': 'stories',
    'chapter': 'chapters'
}

passiveSeed = {
    'carries': 'carried',
    'carry': 'carried',
    'holds': 'held',
    'hold': 'held',
    'takes': 'taken',
    'take': 'taken',
    'brings': 'brought',
    'bring': 'brought',
    'climbs': 'climbed',
    'climb': 'climbed',
    'eats': 'eaten',
    'eat': 'eaten',
    'reads': 'read',
    'read': 'read'
}


In [167]:
it_genderDict = {
    "pera": ["la", "una"],
    "scrittore": ["lo", "uno"],
    "scrittori": ["gli"],
    "banana": ["la", "una"],
    "biscotto": ["il", "un"],
    "libro": ["il", "un"],
    "bottiglia": ["la", "una"],
    "scatola": ["la", "una"],
    "ragazzo": ["il", "un"],
    "ragazzi": ["i"],
    "lampadina": ["la", "una"],
    "credenza": ["la", "una"],
    "cappello": ["il", "un"],
    "gatto": ["il", "un"],
    "gatti": ["i"],
    "capitolo": ["il", "un"],
    "gesso": ["il", "un"],
    "tazza": ["la", "una"],
    "cetriolo": ["il", "un"],
    "cane": ["il", "un"],
    "cani": ["i"],
    "oratorio": ["il", "un"],
    "pesce": ["il", "un"],
    "frutta": ["la", "una"],
    "ragazza": ["la", "una"],
    "ragazze": ["le"],
    "collina": ["la", "una"],
    "uomo": ["l'", "un'"],
    "uomini": ["gli"],
    "pasto": ["il", "un"],
    "montagna": ["la", "una"],
    "topo": ["il", "un"],
    "giornale": ["il", "un"],
    "pizza": ["la", "una"],
    "poeta": ["il", "un"],
    "poeti": ["i"],
    "poema": ["il", "un"],
    "roccia": ["la", "una"],
    "tetto": ["il", "un"],
    "oratore": ["l'", "un'"],
    "oratori": ["gli"],
    "scala": ["la", "una"],
    "storia": ["la", "una"],
    "insegnante": ["l'", "un'"],
    "insegnanti": ["gli"],
    "giocattolo": ["il", "un"],
    "albero": ["l'", "un'"],
    "donna": ["la", "una"],
    "donne": ["le"],
    "autore": ["l'", "un'"],
    "autori": ["gli"]
}

it_pastTense = {
    'scala' : { 'la': 'ha scalata', 'il': 'ha scalato'},
    'legge': { 'la': 'ha letto', 'il': 'ha letto' },
    'porta': { 'la': 'ha portato', 'il': 'ha portato' },
    'mangia': { 'la': 'ha mangiato', 'il': 'ha mangiato' },
    'tiene': { 'la': 'ha tenuto', 'il': 'ha tenuto' },
    'prende' : { 'la': 'ha preso', 'il': 'ha preso'}
}

it_infinitive = {
    'scala' : 'scalare',
    'legge': 'leggere',
    'porta': 'portare',
    'mangia': 'mangiare',
    'tiene': 'tenere',
    'prende' : 'prendere'
}

it_pluralObjects = {
    'pesce': 'pesci',
    'topo': 'topi',
    'bottiglia': 'bottiglie',
    'giornale': 'giornali',
    'gesso': 'gessi',
    'scatola': 'scatole',
    'cappello': 'cappelli',
    'lampadina': 'lampadine',
    'tazza': 'tazze',
    'giocattolo': 'giocattoli',
    'scala': 'scale',
    'roccia': 'rocce',
    'collina': 'colline',
    'montagna': 'montagne',
    'tetto': 'tetti',
    'albero': 'alberi',
    'biscotto': 'biscotti',
    'banana': 'banane',
    'pera': 'pere',
    'pasto': 'pasti',
    'frutta': 'frutta',
    'cetriolo': 'cetrioli',
    'pizza': 'pizze',
    'libro': 'libri',
    'poema': 'poemi',
    'storia': 'storie',
    'capitolo': 'capitolo'
}

it_passiveSeed = {
    'scala' : { 'la': 'è scalata', 'una': 'è scalata', 'il': 'è scalato', 'un': 'è scalato', 'uno': 'è scalato'},
    'legge': { 'la': 'è letto', 'una': 'è letto', 'il': 'è letto' , 'un': 'è letto', 'uno': 'è letto' },
    'porta': { 'la': 'è portato', 'una': 'è portato', 'il': 'è portato' , 'un': 'è portato', 'uno': 'è portato' },
    'mangia': { 'la': 'è mangiato', 'una': 'è mangiato', 'il': 'è mangiato' , 'un': 'è mangiato', 'uno': 'è mangiato' },
    'tiene': { 'la': 'è tenuto', 'una': 'è tenuto', 'il': 'è tenuto' , 'un': 'è tenuto', 'uno': 'è tenuto' },
    'prende' : { 'la': 'è preso', 'una': 'è preso', 'il': 'è preso', 'un': 'è preso', 'uno': 'è preso'},
    'salgono' : { 'la': 'è scalata', 'una': 'è scalata', 'il': 'è scalato', 'un': 'è scalato', 'uno': 'è scalato'},
    'leggono': { 'la': 'è letto', 'una': 'è letto', 'il': 'è letto' , 'un': 'è letto', 'uno': 'è letto' },
    'portano': { 'la': 'è portato', 'una': 'è portato', 'il': 'è portato' , 'un': 'è portato', 'uno': 'è portato' },
    'mangiano': { 'la': 'è mangiato', 'una': 'è mangiato', 'il': 'è mangiato' , 'un': 'è mangiato', 'uno': 'è mangiato' },
    'tengono': { 'la': 'è tenuto', 'una': 'è tenuto', 'il': 'è tenuto' , 'un': 'è tenuto', 'uno': 'è tenuto' },
    'prendono' : { 'la': 'è preso', 'una': 'è preso', 'il': 'è preso', 'un': 'è preso', 'uno': 'è preso'}
}


# Generate sentences

In [205]:
seed = [ { 'verb' : ['carries', 'holds', 'takes', 'brings'],  'subject': ['dog', 'cat', 'man', 'woman', 'teacher', 'girl', 'boy'], 'object': ['fish', 'mouse', 'bottle', 'newspaper', 'chalk', 'box', 'cap', 'bulb', 'cup', 'toy']},

{ 'verb': ['climbs'], 'subject': ['dog', 'cat', 'man', 'woman', 'teacher', 'girl', 'boy'], 'object': ['staircase', 'rock', 'hill', 'mountain', 'roof', 'tree'] },

{ 'verb': ['eats'], 'subject' : ['dog', 'cat', 'man', 'woman', 'teacher', 'girl', 'boy'], 'object': ['biscuit', 'fish', 'banana', 'pear', 'meal', 'fruit', 'cucumber', 'pizza' ]},
{'verb': ['reads'], 'subject' : ['poet', 'author', 'writer', 'speaker', 'teacher', 'girl', 'boy'], 'object': ['book', 'poem', 'story', 'chapter']} ]

subordinateSeed = [ { 'verb' : ['sees', 'says', 'notices', 'states', 'claims'],  'subject': ['Sheela', 'Leela', 'Maria', 'Gomu', 'John', 'Tom', 'Harry'], }]


it_seed = [ { 'verb' : ['porta', 'tiene', 'prende', 'porta'],  'subject': ['cane', 'gatto', 'uomo', 'donna', 'insegnante', 'ragazza', 'ragazzo'], 'object': ['pesce', 'topo', 'bottiglia', 'giornale', 'gesso', 'scatola', 'cappello', 'lampadina', 'tazza', 'giocattolo']},

{ 'verb': ['scala'], 'subject': ['cane', 'gatto', 'uomo', 'donna', 'insegnante', 'ragazza', 'ragazzo'], 'object': ['scala', 'roccia', 'collina', 'montagna', 'tetto', 'albero'] },

{ 'verb': ['mangia'], 'subject' : ['cane', 'gatto', 'uomo', 'donna', 'insegnante', 'ragazza', 'ragazzo'], 'object': ['biscotto', 'pesce', 'banana', 'pera', 'pasto', 'frutta', 'cetriolo', 'pizza' ]},
{'verb': ['legge'], 'subject' : ['poeta', 'autore', 'scrittore', 'oratorio', 'insegnante', 'ragazza', 'ragazzo'], 'object': ['libro', 'poema', 'storia', 'capitolo']} ]

it_subordinateSeed = [ { 'verb' : ['vede', 'dice', 'osserva', 'sa', 'afferma'],  'subject': ['Sheela', 'Leela', 'Maria', 'Gomu', 'John', 'Tom', 'Harry'], }]

df = pd.DataFrame()

import random
for oidx, obj in enumerate(seed):
    for sidx, subj in enumerate(obj['subject']):
        for obidx, ob in enumerate(obj['object']):
            for vidx, verb in enumerate(obj['verb']):
                sdet = random.choice(['the', 'a'])
                odet = random.choice(['the', 'a'])
                pSubj = random.choice(subordinateSeed[0]['subject'])
                pVerb = random.choice(subordinateSeed[0]['verb'])

                if sdet == 'the':
                    it_sdet = it_genderDict[it_seed[oidx]['subject'][sidx]][0] 
                else: 
                    it_sdet = it_genderDict[it_seed[oidx]['subject'][sidx]][1]

                if odet == 'the':
                    it_odet = it_genderDict[it_seed[oidx]['object'][obidx]][0] 
                else: 
                    it_odet = it_genderDict[it_seed[oidx]['object'][obidx]][1]
                
                it_subj = it_seed[oidx]['subject'][sidx]
                it_ob = it_seed[oidx]['object'][obidx]
                it_verb = it_seed[oidx]['verb'][vidx]
                
                it_pSubj = random.choice(it_subordinateSeed[0]['subject'])
                it_pVerb = random.choice(it_subordinateSeed[0]['verb'])

                temp_odet = it_odet
                if it_odet =='il':
                    ita_passive_from = 'dal'
                if it_odet =='la':
                    ita_passive_from = 'dalla'
                elif it_odet == "un" or it_odet == "un'":
                    ita_passive_from = 'da un'
                    temp_odet = 'un'
                elif it_odet == "una":
                    ita_passive_from = 'da una'
                elif it_odet == "l'":
                    temp_odet = 'il'
                    ita_passive_from = "dall'"
                ita_passive_sentence = f"{it_odet} {it_ob} {it_passiveSeed[it_verb][temp_odet]} {ita_passive_from} {it_subj}"

                if (it_sdet != "l'" and it_odet != "l'"):
                    df = pd.concat([df, pd.DataFrame.from_dict([{
                        "ita": f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}",
                        "ita-r-1-null_subject": f"{it_verb} {it_odet} {it_ob}",
                        "ita-r-2-subordinate": f"{it_pSubj} {it_pVerb} che {it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}",
                        "ita-r-3-passive":ita_passive_sentence,
                        "ita-u-1-negation": f"{it_sdet} {it_subj} {it_verb} {it_odet} no {it_ob}",
                        "ita-u-2-invert": " ".join(f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}".split(" ")[::-1]),
                        "ita-u-3-gender":f"{it_odet} {it_subj} {it_verb} {it_odet} {it_ob}",
                        "en-r-1-sentence": f"{sdet} {subj} {verb} {odet} {ob}",
                        "en-r-2-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                        "en-r-3-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}"}])]),
                elif (it_sdet == "l'" and it_odet != "l'"):
                    df = pd.concat([df, pd.DataFrame.from_dict([{
                        "ita": f"{it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}",
                        "ita-r-1-null_subject": f"{it_verb} {it_odet} {it_ob}",
                        "ita-r-2-subordinate": f"{it_pSubj} {it_pVerb} che {it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}",
                        "ita-r-3-passive":ita_passive_sentence,
                        "ita-u-1-negation": f"{it_sdet}{it_subj} {it_verb} {it_odet} no {it_ob}",
                        "ita-u-2-invert": " ".join(f"{it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}".split(" ")[::-1]),
                        "ita-u-3-gender": f"{it_sdet}{it_subj} {it_verb} {it_odet} {it_ob}", #using sdet instead of odet for subj, because it's a word starting w a vowel, and therefore the det is not gendered
                        "en-r-1-sentence": f"{sdet} {subj} {verb} {odet} {ob}",
                        "en-r-2-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                        "en-r-3-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}"}])])
                elif (it_sdet == "l'" and it_odet == "l'"):
                    df = pd.concat([df, pd.DataFrame.from_dict([{
                        "ita": f"{it_sdet}{it_subj} {it_verb} {it_odet}{it_ob}",
                        "ita-r-1-null_subject": f"{it_verb} {it_odet}{it_ob}",
                        "ita-r-2-subordinate": f"{it_pSubj} {it_pVerb} che {it_sdet} {it_subj} {it_verb} {it_odet}{it_ob}",
                        "ita-r-3-passive":ita_passive_sentence, 
                        "ita-u-1-negation": f"{it_sdet}{it_subj} {it_verb} il no {it_ob}",
                        "ita-u-2-invert": " ".join( f"{it_sdet}{it_subj} {it_verb} {it_odet}{it_ob}".split(" ")[::-1]),
                        "ita-u-3-gender": f"{it_odet}{it_subj} {it_verb} {it_odet}{it_ob}", 
                        "en-r-1-sentence": f"{sdet} {subj} {verb} {odet} {ob}", 
                        "en-r-2-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}", 
                        "en-r-3-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}"}])])
                else:
                    df = pd.concat([df, pd.DataFrame.from_dict([{
                        "ita": f"{it_sdet} {it_subj} {it_verb} {it_odet}{it_ob}",
                        "ita-r-1-null_subject": f"{it_verb} {it_odet}{it_ob}",
                        "ita-r-2-subordinate": f"{it_pSubj} {it_pVerb} che {it_sdet}{it_subj} {it_verb} {it_odet}{it_ob}",
                        "ita-r-3-passive":ita_passive_sentence, 
                        "ita-u-1-negation": f"{it_sdet} {it_subj} {it_verb} {it_odet} no {it_ob}",
                        "ita-u-2-invert": " ".join(f"{it_sdet} {it_subj} {it_verb} {it_odet}{it_ob}".split(" ")[::-1]),
                        "ita-u-3-gender": f"il {it_subj} {it_verb} {it_odet}{it_ob}",
                        "en-r-1-sentence": f"{sdet} {subj} {verb} {odet} {ob}",
                        "en-r-2-subordinate": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                        "en-r-3-passive": f"{odet} {ob} is {passiveSeed[verb]} by {sdet} {subj}", }])])
                

TypeError: cannot concatenate object of type '<class 'tuple'>'; only Series and DataFrame objs are valid

In [186]:

pluralSeed = [ { 'verb' : ['carry', 'hold', 'take', 'bring'],  'subject': ['dogs', 'cats', 'men', 'women', 'teachers', 'girls', 'boys'], 'object': ['fish', 'mouse', 'bottle', 'newspaper', 'chalk', 'box', 'cap', 'bulb', 'cup', 'toy']},

{ 'verb': ['climb'], 'subject': ['dogs', 'cats', 'men', 'women', 'teachers', 'girls', 'boys'], 'object': ['staircase', 'rock', 'hill', 'mountain', 'roof', 'tree'] },

{ 'verb': ['eat'], 'subject' : ['dogs', 'cats', 'men', 'women', 'teachers', 'girls', 'boys'], 'object': ['biscuit', 'fish', 'banana', 'pear', 'meal', 'fruit', 'cucumber', 'pizza' ] },
{'verb': ['read'], 'subject' : ['poets', 'authors', 'writers', 'speakers', 'teachers', 'girls', 'boys'], 'object': ['book', 'poem', 'story', 'chapter']} ]

it_pluralSeed = [ { 'verb' : ['portano', 'tengono', 'prendono', 'portano'],  'subject': ['cani', 'gatti', 'uomini', 'donne', 'insegnanti', 'ragazze', 'ragazzi'], 'object': ['pesce', 'topo', 'bottiglia', 'giornale', 'gesso', 'scatola', 'cappello', 'lampadina', 'tazza', 'giocattolo']},

{ 'verb': ['salgono'], 'subject': ['cani', 'gatti', 'uomini', 'donne', 'insegnanti', 'ragazze', 'ragazzi'], 'object': ['scala', 'roccia', 'collina', 'montagna', 'tetto', 'albero'] },

{ 'verb': ['mangiano'], 'subject' : ['cani', 'gatti', 'uomini', 'donne', 'insegnanti', 'ragazze', 'ragazzi'], 'object': ['biscotto', 'pesce', 'banana', 'pera', 'pasto', 'frutta', 'cetriolo', 'pizza' ] },
{'verb': ['leggono'], 'subject' : ['poeti', 'autori', 'scrittori', 'oratori', 'insegnanti', 'ragazze', 'ragazzi'], 'object': ['libro', 'poema', 'storia', 'capitolo']} ]

it_subordinateSeed = [ { 'verb' : ['vede', 'dice', 'osserva', 'sa', 'afferma'],  'subject': ['Sheela', 'Leela', 'Maria', 'Gomu', 'John', 'Tom', 'Harry'], }]

import random
for oidx, obj in enumerate(pluralSeed):
    for sidx, subj in enumerate(obj['subject']):
        for obidx, ob in enumerate(obj['object']):
            for vidx, verb in enumerate(obj['verb']):
                pSubj = random.choice(subordinateSeed[0]['subject'])
                pVerb = random.choice(subordinateSeed[0]['verb'])
                odet = random.choice(['the', 'a'])
                
                it_subj = it_pluralSeed[oidx]['subject'][sidx]
                it_sdet = it_genderDict[it_subj][0]
                it_ob = it_pluralSeed[oidx]['object'][obidx]
                it_odet = it_genderDict[it_ob][0]

                it_verb = it_pluralSeed[oidx]['verb'][vidx]
                
                it_pSubj = random.choice(it_subordinateSeed[0]['subject'])
                it_pVerb = random.choice(it_subordinateSeed[0]['verb'])

                temp_sdet = it_sdet
                if it_sdet == 'i':
                    temp_sdet = 'il'
                    ita_passive_from = 'dai'
                if it_sdet == 'le':
                    temp_sdet = 'la'
                    ita_passive_from = 'dalle'
                elif it_sdet == "gli":
                    temp_sdet = 'il'
                    ita_passive_from = "dagli'"
                ita_passive_sentence = f"{it_odet} {it_ob} {it_passiveSeed[it_verb][temp_sdet]} {ita_passive_from} {it_subj}"

                if it_odet == 'l':
                    temp_odet = 'il'
                else:
                    temp_odet = it_odet
                    
                df = pd.concat([df, pd.DataFrame.from_dict([{
                    "ita": f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}", 
                    "ita-r-1-null_subject": f"{it_verb} {it_odet} {it_ob}", 
                    "ita-r-2-subordinate": f"{it_pSubj} {it_pVerb} che {it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}",
                    "ita-r-3-passive": ita_passive_sentence,
                    "ita-u-1-negation": f"{it_sdet} {it_subj} {it_verb} {it_odet} no {it_ob}",
                    "ita-u-2-invert": " ".join(f"{it_sdet} {it_subj} {it_verb} {it_odet} {it_ob}".split(" ")[::-1]), 
                    "ita-u-3-gender": f"{temp_odet} {it_subj} {it_verb} {it_odet} {it_ob}",
                    "sentence": f"the {subj} {verb} {odet} {ob}",
                    "subordinate-sentence": f"{pSubj} {pVerb} that the {subj} {verb} the {ob}",
                    "passive-sentence": f"{odet} {ob} is {passiveSeed[verb]} by the {subj}"}])]),
                    "en-u-1-negation": f"{sdet} {subj} {verb} {odet} doesn't {ob}",
                    "en-u-2-inversion": " ".join(f"{sdet} {subj} {verb} {odet} {ob}".split(" ")[::-1]),
                    "en-u-3-qsubordinate": f"{pSubj} {pVerb} that does the {subj} {verb} the {ob}?"
                if (f"{odet} {ob} is {passiveSeed[verb]} by the {subj}" == None):
                    print(f"{odet} {ob} is {passiveSeed[verb]} by the {subj}", odet, ob, passiveSeed[verb], subj)

df.reset_index()

IndentationError: unexpected indent (837266882.py, line 64)

## Italian sentence

In [158]:
df['it'] = df.progress_apply(lambda row: " ".join([genderDict[row['sentence'].split(" ")[1]]] + row['sentence'].split(" ")[1:3] + [genderDict[row['sentence'].split(" ")[4]]] + [row['sentence'].split(" ")[-1]]), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 85915.46it/s]


## IT Real grammar 1 (Null Subject parameter)

In [159]:
df['it-r-1-null_subject'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[2:]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 120690.84it/s]


## IT Real Grammar 2 (Passive construction)

In [160]:
df['it-r-2-passive'] =  df.progress_apply(lambda row: " ".join([genderDict[row['passive-sentence'].split(" ")[1]]] + row['passive-sentence'].split(" ")[1:-2] + [genderDict[row['passive-sentence'].split(" ")[-1]]] + [row['passive-sentence'].split(" ")[-1]]), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 80684.53it/s]


## IT Real Grammar 3 (Subordinate construction)

In [161]:
df['it-r-3-subordinate'] =  df.progress_apply(lambda row: " ".join(row['subordinate-sentence'].split(" ")[0:3] + row['it'].split(" ")), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 110760.51it/s]


In [54]:
df

,sentence,subordinate-sentence,passive-sentence,it,it-r-1-null_subject,it-r-2-passive,it-r-3-subordinate
0,a dog carries a fish,Sheela claims that the dog carries the fish,a fish is carried by a dog,kon dog carries kon fish,carries kon fish,kon fish is carried by kon dog,Sheela claims that kon dog carries kon fish
0,a dog holds the fish,John sees that the dog holds the fish,the fish is held by a dog,kon dog holds kon fish,holds kon fish,kon fish is held by kon dog,John sees that kon dog holds kon fish
0,the dog takes the fish,Leela notices that the dog takes the fish,the fish is taken by the dog,kon dog takes kon fish,takes kon fish,kon fish is taken by kon dog,Leela notices that kon dog takes kon fish
0,the dog brings a fish,John notices that the dog brings the fish,a fish is brought by the dog,kon dog brings kon fish,brings kon fish,kon fish is brought by kon dog,John notices that kon dog brings kon fish
0,a dog carries the mouse,Maria states that the dog carries the mouse,the mouse is carried by a dog,kon dog carries kon mouse,carries kon mouse,kon mouse is carried by kon dog,Maria states that kon dog carries kon mouse
...,...,...,...,...,...,...,...
0,the girls read the chapter,Harry claims that the girls read the chapter,the chapter is read by the girls,kars girls read kon chapter,read kon chapter,kon chapter is read by kars girls,Harry claims that kars girls read kon chapter
0,the boys read the book,Tom states that the boys read the book,the book is read by the boys,kons boys read kon book,read kon book,kon book is read by kons boys,Tom states that kons boys read kon book
0,the boys read the poem,John says that the boys read the poem,the poem is read by the boys,kons boys read kar poem,read kar poem,kar poem is read by kons boys,John says that kons boys read kar poem
0,the boys read a story,Harry states that the boys read the story,a story is read by the boys,kons boys read kar story,read kar story,kar story is read by kons boys,Harry states that kons boys read kar story


## IT Unreal Grammar 1: Add a negation after the 3rd word in the nullified subject sentence

In [55]:
df['it-u-1-negation'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[:4] + [ "no" ] + row['sentence'].split(" ")[4:]), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 90653.86it/s]


## IT Unreal Grammar 2: Invert italian sentence

In [56]:
df['it-u-2-invert'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[::-1]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 126717.08it/s]


## IT Unreal Grammar 3: Same gender for subject and object

In [57]:
df['it-u-3-gender'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[:3] + [row['it'].split(" ")[0]] + [row['it'].split(" ")[-1]]), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 95221.15it/s]


## JP real grammar 1 (Wa after subj, o after obj, verb)

In [58]:
df['jp-r-1-sov'] = df.progress_apply(lambda row: " ".join(row['sentence'].split(" ")[:2]) + " wa " + " ".join(row['sentence'].split(" ")[-2:]) + " o " + row['sentence'].split(" ")[2], axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 102887.28it/s]


## JP real grammar 2 (Passive construction)

In [59]:
df['jp-r-2-passive'] = df.progress_apply(lambda row: " ".join(row['sentence'].split(" ")[3:5]) + " wa " + " ".join(row['sentence'].split(" ")[:2]) + " ni " + infinitive[row['sentence'].split(" ")[2]] + " reru", axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 94045.81it/s]


In [60]:
df

,sentence,subordinate-sentence,passive-sentence,it,it-r-1-null_subject,it-r-2-passive,it-r-3-subordinate,it-u-1-negation,it-u-2-invert,it-u-3-gender,jp-r-1-sov,jp-r-2-passive
0,a dog carries a fish,Sheela claims that the dog carries the fish,a fish is carried by a dog,kon dog carries kon fish,carries kon fish,kon fish is carried by kon dog,Sheela claims that kon dog carries kon fish,kon dog carries kon no fish,fish kon carries dog kon,kon dog carries kon fish,a dog wa a fish o carries,a fish wa a dog ni to carry reru
0,a dog holds the fish,John sees that the dog holds the fish,the fish is held by a dog,kon dog holds kon fish,holds kon fish,kon fish is held by kon dog,John sees that kon dog holds kon fish,kon dog holds kon no fish,fish kon holds dog kon,kon dog holds kon fish,a dog wa the fish o holds,the fish wa a dog ni to hold reru
0,the dog takes the fish,Leela notices that the dog takes the fish,the fish is taken by the dog,kon dog takes kon fish,takes kon fish,kon fish is taken by kon dog,Leela notices that kon dog takes kon fish,kon dog takes kon no fish,fish kon takes dog kon,kon dog takes kon fish,the dog wa the fish o takes,the fish wa the dog ni to take reru
0,the dog brings a fish,John notices that the dog brings the fish,a fish is brought by the dog,kon dog brings kon fish,brings kon fish,kon fish is brought by kon dog,John notices that kon dog brings kon fish,kon dog brings kon no fish,fish kon brings dog kon,kon dog brings kon fish,the dog wa a fish o brings,a fish wa the dog ni to bring reru
0,a dog carries the mouse,Maria states that the dog carries the mouse,the mouse is carried by a dog,kon dog carries kon mouse,carries kon mouse,kon mouse is carried by kon dog,Maria states that kon dog carries kon mouse,kon dog carries kon no mouse,mouse kon carries dog kon,kon dog carries kon mouse,a dog wa the mouse o carries,the mouse wa a dog ni to carry reru
...,...,...,...,...,...,...,...,...,...,...,...,...
0,the girls read the chapter,Harry claims that the girls read the chapter,the chapter is read by the girls,kars girls read kon chapter,read kon chapter,kon chapter is read by kars girls,Harry claims that kars girls read kon chapter,kars girls read kon no chapter,chapter kon read girls kars,kars girls read kars chapter,the girls wa the chapter o read,the chapter wa the girls ni to read reru
0,the boys read the book,Tom states that the boys read the book,the book is read by the boys,kons boys read kon book,read kon book,kon book is read by kons boys,Tom states that kons boys read kon book,kons boys read kon no book,book kon read boys kons,kons boys read kons book,the boys wa the book o read,the book wa the boys ni to read reru
0,the boys read the poem,John says that the boys read the poem,the poem is read by the boys,kons boys read kar poem,read kar poem,kar poem is read by kons boys,John says that kons boys read kar poem,kons boys read kar no poem,poem kar read boys kons,kons boys read kons poem,the boys wa the poem o read,the poem wa the boys ni to read reru
0,the boys read a story,Harry states that the boys read the story,a story is read by the boys,kons boys read kar story,read kar story,kar story is read by kons boys,Harry states that kons boys read kar story,kons boys read kar no story,story kar read boys kons,kons boys read kons story,the boys wa a story o read,a story wa the boys ni to read reru


## JP real grammar 3 (Subordinate construction)

In [61]:
df['jp-r-3-subordinate'] = df.progress_apply(lambda row: " ".join([row['subordinate-sentence'].split(" ")[0]] + ["wa"] + row['subordinate-sentence'].split(" ")[3:5] + ["ga"] + row['subordinate-sentence'].split(" ")[-2:]  + ["o"] + [row['subordinate-sentence'].split(" ")[5]] + ["to"] + [row['subordinate-sentence'].split(" ")[1]]), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 80659.69it/s]


## JP - Unreal grammar 1:Add a negation at the end of the object in the real-jp-1 sentence

In [62]:
df['jp-u-1-negation'] = df.progress_apply(lambda row: " ".join(row['jp-r-1-sov'].split(" ")[:3]) + " no " + " ".join(row['jp-r-1-sov'].split(" ")[3:]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 107204.35it/s]


## JP - Unreal grammar 2: Invert jp-real-1 sentence

In [63]:
df['jp-u-2-invert'] = df.progress_apply(lambda row: " ".join(row['jp-r-1-sov'].split(" ")[::-1]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 119572.20it/s]


In [64]:
df

,sentence,subordinate-sentence,passive-sentence,it,it-r-1-null_subject,it-r-2-passive,it-r-3-subordinate,it-u-1-negation,it-u-2-invert,it-u-3-gender,jp-r-1-sov,jp-r-2-passive,jp-r-3-subordinate,jp-u-1-negation,jp-u-2-invert
0,a dog carries a fish,Sheela claims that the dog carries the fish,a fish is carried by a dog,kon dog carries kon fish,carries kon fish,kon fish is carried by kon dog,Sheela claims that kon dog carries kon fish,kon dog carries kon no fish,fish kon carries dog kon,kon dog carries kon fish,a dog wa a fish o carries,a fish wa a dog ni to carry reru,Sheela wa the dog ga the fish o carries to claims,a dog wa no a fish o carries,carries o fish a wa dog a
0,a dog holds the fish,John sees that the dog holds the fish,the fish is held by a dog,kon dog holds kon fish,holds kon fish,kon fish is held by kon dog,John sees that kon dog holds kon fish,kon dog holds kon no fish,fish kon holds dog kon,kon dog holds kon fish,a dog wa the fish o holds,the fish wa a dog ni to hold reru,John wa the dog ga the fish o holds to sees,a dog wa no the fish o holds,holds o fish the wa dog a
0,the dog takes the fish,Leela notices that the dog takes the fish,the fish is taken by the dog,kon dog takes kon fish,takes kon fish,kon fish is taken by kon dog,Leela notices that kon dog takes kon fish,kon dog takes kon no fish,fish kon takes dog kon,kon dog takes kon fish,the dog wa the fish o takes,the fish wa the dog ni to take reru,Leela wa the dog ga the fish o takes to notices,the dog wa no the fish o takes,takes o fish the wa dog the
0,the dog brings a fish,John notices that the dog brings the fish,a fish is brought by the dog,kon dog brings kon fish,brings kon fish,kon fish is brought by kon dog,John notices that kon dog brings kon fish,kon dog brings kon no fish,fish kon brings dog kon,kon dog brings kon fish,the dog wa a fish o brings,a fish wa the dog ni to bring reru,John wa the dog ga the fish o brings to notices,the dog wa no a fish o brings,brings o fish a wa dog the
0,a dog carries the mouse,Maria states that the dog carries the mouse,the mouse is carried by a dog,kon dog carries kon mouse,carries kon mouse,kon mouse is carried by kon dog,Maria states that kon dog carries kon mouse,kon dog carries kon no mouse,mouse kon carries dog kon,kon dog carries kon mouse,a dog wa the mouse o carries,the mouse wa a dog ni to carry reru,Maria wa the dog ga the mouse o carries to states,a dog wa no the mouse o carries,carries o mouse the wa dog a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,the girls read the chapter,Harry claims that the girls read the chapter,the chapter is read by the girls,kars girls read kon chapter,read kon chapter,kon chapter is read by kars girls,Harry claims that kars girls read kon chapter,kars girls read kon no chapter,chapter kon read girls kars,kars girls read kars chapter,the girls wa the chapter o read,the chapter wa the girls ni to read reru,Harry wa the girls ga the chapter o read to cl...,the girls wa no the chapter o read,read o chapter the wa girls the
0,the boys read the book,Tom states that the boys read the book,the book is read by the boys,kons boys read kon book,read kon book,kon book is read by kons boys,Tom states that kons boys read kon book,kons boys read kon no book,book kon read boys kons,kons boys read kons book,the boys wa the book o read,the book wa the boys ni to read reru,Tom wa the boys ga the book o read to states,the boys wa no the book o read,read o book the wa boys the
0,the boys read the poem,John says that the boys read the poem,the poem is read by the boys,kons boys read kar poem,read kar poem,kar poem is read by kons boys,John says that kons boys read kar poem,kons boys read kar no poem,poem kar read boys kons,kons boys read kons poem,the boys wa the poem o read,the poem wa the boys ni to read reru,John wa the boys ga the poem o read to says,the boys wa no the poem o read,read o poem the wa boys the
0,the boys read a story,Harry states that the boys read the sto

## JP - Unreal grammar add a after o + past tense

In [65]:
df['jp-u-3-past-tense'] = df.progress_apply(lambda row: " ".join(row['jp-r-1-sov'].split(" ")[:-2] + [' o-ta '] + [infinitive[row['jp-r-1-sov'].split(" ")[-1]]]), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 98486.88it/s]


In [66]:
df

,sentence,subordinate-sentence,passive-sentence,it,it-r-1-null_subject,it-r-2-passive,it-r-3-subordinate,it-u-1-negation,it-u-2-invert,it-u-3-gender,jp-r-1-sov,jp-r-2-passive,jp-r-3-subordinate,jp-u-1-negation,jp-u-2-invert,jp-u-3-past-tense
0,a dog carries a fish,Sheela claims that the dog carries the fish,a fish is carried by a dog,kon dog carries kon fish,carries kon fish,kon fish is carried by kon dog,Sheela claims that kon dog carries kon fish,kon dog carries kon no fish,fish kon carries dog kon,kon dog carries kon fish,a dog wa a fish o carries,a fish wa a dog ni to carry reru,Sheela wa the dog ga the fish o carries to claims,a dog wa no a fish o carries,carries o fish a wa dog a,a dog wa a fish o-ta to carry
0,a dog holds the fish,John sees that the dog holds the fish,the fish is held by a dog,kon dog holds kon fish,holds kon fish,kon fish is held by kon dog,John sees that kon dog holds kon fish,kon dog holds kon no fish,fish kon holds dog kon,kon dog holds kon fish,a dog wa the fish o holds,the fish wa a dog ni to hold reru,John wa the dog ga the fish o holds to sees,a dog wa no the fish o holds,holds o fish the wa dog a,a dog wa the fish o-ta to hold
0,the dog takes the fish,Leela notices that the dog takes the fish,the fish is taken by the dog,kon dog takes kon fish,takes kon fish,kon fish is taken by kon dog,Leela notices that kon dog takes kon fish,kon dog takes kon no fish,fish kon takes dog kon,kon dog takes kon fish,the dog wa the fish o takes,the fish wa the dog ni to take reru,Leela wa the dog ga the fish o takes to notices,the dog wa no the fish o takes,takes o fish the wa dog the,the dog wa the fish o-ta to take
0,the dog brings a fish,John notices that the dog brings the fish,a fish is brought by the dog,kon dog brings kon fish,brings kon fish,kon fish is brought by kon dog,John notices that kon dog brings kon fish,kon dog brings kon no fish,fish kon brings dog kon,kon dog brings kon fish,the dog wa a fish o brings,a fish wa the dog ni to bring reru,John wa the dog ga the fish o brings to notices,the dog wa no a fish o brings,brings o fish a wa dog the,the dog wa a fish o-ta to bring
0,a dog carries the mouse,Maria states that the dog carries the mouse,the mouse is carried by a dog,kon dog carries kon mouse,carries kon mouse,kon mouse is carried by kon dog,Maria states that kon dog carries kon mouse,kon dog carries kon no mouse,mouse kon carries dog kon,kon dog carries kon mouse,a dog wa the mouse o carries,the mouse wa a dog ni to carry reru,Maria wa the dog ga the mouse o carries to states,a dog wa no the mouse o carries,carries o mouse the wa dog a,a dog wa the mouse o-ta to carry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,the girls read the chapter,Harry claims that the girls read the chapter,the chapter is read by the girls,kars girls read kon chapter,read kon chapter,kon chapter is read by kars girls,Harry claims that kars girls read kon chapter,kars girls read kon no chapter,chapter kon read girls kars,kars girls read kars chapter,the girls wa the chapter o read,the chapter wa the girls ni to read reru,Harry wa the girls ga the chapter o read to cl...,the girls wa no the chapter o read,read o chapter the wa girls the,the girls wa the chapter o-ta to read
0,the boys read the book,Tom states that the boys read the book,the book is read by the boys,kons boys read kon book,read kon book,kon book is read by kons boys,Tom states that kons boys read kon book,kons boys read kon no book,book kon read boys kons,kons boys read kons book,the boys wa the book o read,the book wa the boys ni to read reru,Tom wa the boys ga the book o read to states,the boys wa no the book o read,read o book the wa boys the,the boys wa the book o-ta to read
0,the boys read the poem,John says that the boys read the poem,the poem is read by the boys,kons boys read kar poem,read kar poem,kar poem is read by kons boys,John says that kons boys read kar poem,kons boys read kar no poem,poem kar read boys kons,kon

## Non-grammatical sentences

In [67]:
swapProhibited = { 
'sentence': [ [0,3], [1,4] ],
'subordinate-sentence': [ [0,4], [0,7], [4,7], [3,6] ],
'passive-sentence': [ [0,5], [1,6] ],
'it': [ [0,3], [1,4] ],
'it-r-1-null_subject': [ ],
'it-r-2-passive': [ [0,5], [1,6] ],
'it-r-3-subordinate': [ [0,5], [1,6] ],
'it-u-1-negation': [ [0,3], [1,5] ],
'it-u-2-invert': [ [0,3], [1,4] ],
'it-u-3-gender': [ [0,3], [1,4] ],
'jp-r-1-sov': [ [0,3], [1,4] ],
'jp-r-2-passive': [ [0,3], [1,4] ],
'jp-r-3-subordinate': [ [0,3], [0,6], [3,6] ],
'jp-u-1-negation': [ [0,4], [1,5] ],
'jp-u-2-invert': [ [3,6], [2,5] ],
'jp-u-3-past-tense': [ [0,3], [1,4] ],
}
def swap_words(sentence, col):
    sentence = sentence.split(" ")
    numWords = len(sentence)
    toSwap = random.sample(range(0, numWords), 2)
    toSwapWords = set([sentence[toSwap[0]],sentence[toSwap[1]]])
    prohibitedWords = [set([sentence[pos[0]],sentence[pos[1]]]) for pos in swapProhibited[col]]
    while toSwapWords in prohibitedWords:
        toSwap = random.sample(range(0, numWords), 2)
        toSwapWords = set([sentence[toSwap[0]],sentence[toSwap[1]]])
    
    swap1 = sentence[toSwap[0]]
    swap2 = sentence[toSwap[1]]
    sentence[toSwap[0]] = swap2
    sentence[toSwap[1]] = swap1
    return " ".join(sentence)

for col in swapProhibited.keys():
    print(' Now processing.... ', col)
    df[f'ng-{col}'] = df.progress_apply(lambda row: swap_words(row[col], col), axis=1)

 Now processing....  sentence


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 126088.44it/s]


 Now processing....  subordinate-sentence


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 123594.67it/s]


 Now processing....  passive-sentence


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 131629.24it/s]


 Now processing....  jp-u-2-invert


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 137534.82it/s]


 Now processing....  jp-u-3-past-tense


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 812/812 [00:00<00:00, 115850.56it/s]


In [68]:
gCols = [col for col in df.columns if not 'ng' in col]
ngCols = [col for col in df.columns if 'ng' in col]
df[ngCols]

,ng-sentence,ng-subordinate-sentence,ng-passive-sentence,ng-it,ng-it-r-1-null_subject,ng-it-r-2-passive,ng-it-r-3-subordinate,ng-it-u-1-negation,ng-it-u-2-invert,ng-it-u-3-gender,ng-jp-r-1-sov,ng-jp-r-2-passive,ng-jp-r-3-subordinate,ng-jp-u-1-negation,ng-jp-u-2-invert,ng-jp-u-3-past-tense
0,dog a carries a fish,carries claims that the dog Sheela the fish,a fish carried is by a dog,kon dog kon carries fish,carries fish kon,kon carried is fish by kon dog,kon claims that Sheela dog carries kon fish,kon dog carries fish no kon,kon fish carries dog kon,kon carries dog kon fish,a dog carries a fish o wa,to fish wa a dog ni a carry reru,wa Sheela the dog ga the fish o carries to claims,carries dog wa no a fish o a,carries a fish o wa dog a,a dog wa a fish o-ta to carry
0,dog a holds the fish,John sees holds the dog that the fish,the is fish held by a dog,kon dog fish kon holds,kon holds fish,dog fish is held by kon kon,John holds that kon dog sees kon fish,kon dog holds fish no kon,fish kon kon dog holds,dog kon holds kon fish,a dog wa holds fish o the,the fish hold a dog ni to wa reru,John wa dog the ga the fish o holds to sees,a dog no wa the fish o holds,holds o fish dog wa the a,hold dog wa the fish o-ta to a
0,fish dog takes the the,that notices Leela the dog takes the fish,the taken is fish by the dog,kon kon takes dog fish,kon takes fish,fish kon is taken by kon dog,Leela that notices kon dog takes kon fish,kon dog no kon takes fish,kon kon takes dog fish,kon dog fish kon takes,wa dog the the fish o takes,the fish wa the dog ni reru take to,Leela wa the dog o the fish ga takes to notices,takes dog wa no the fish o the,fish o takes the wa dog the,the dog wa take fish o-ta to the
0,dog the brings a fish,the notices that the dog brings John fish,a fish dog brought by the is,kon kon brings dog fish,fish kon brings,kon fish is dog by kon brought,John notices that kon dog kon brings fish,kon kon brings dog no fish,fish kon brings kon dog,kon dog kon brings fish,the wa dog a fish o brings,a fish wa the dog ni bring to reru,ga wa the dog John the fish o brings to notices,the dog wa o a fish no brings,a o fish brings wa dog the,the dog to a fish o-ta wa bring
0,mouse dog carries the a,Maria states the the dog carries that mouse,the carried is mouse by a dog,kon carries dog kon mouse,mouse kon carries,kon mouse is carried by dog kon,Maria states kon kon dog carries that mouse,kon no carries kon dog mouse,mouse kon carries kon dog,kon dog carries mouse kon,a dog carries the mouse o wa,mouse the wa a dog ni to carry reru,Maria wa the dog the ga mouse o carries to states,a dog wa no mouse the o carries,mouse o carries the wa dog a,wa dog a the mouse o-ta to carry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,the girls read chapter the,Harry that claims the girls read the chapter,the chapter by read is the girls,kars girls read chapter kon,chapter kon read,read chapter is kon by kars girls,Harry claims that girls kars read kon chapter,kars girls read no kon chapter,chapter kon girls read kars,kars kars read girls chapter,the girls wa the o chapter read,the reru wa the girls ni to read chapter,Harry wa claims girls ga the chapter o read to...,the girls wa no the o chapter read,read o the the wa girls chapter,the to wa the chapter o-ta girls read
0,the the read boys book,Tom book that the boys read the states,the book boys read by the is,kons kon read boys book,kon read book,kon book by read is kons boys,that states Tom kons boys read kon book,kons read boys kon no book,kons kon read boys book,kons boys read book kons,the the wa boys book o read,the book ni the boys wa to read reru,Tom to the boys ga the book o read wa states,the boys wa the no book o read,read the book o wa boys the,boys the wa the book o-ta to read
0,the the read boys poem,John read that the boys says the poem,the poem is by read the boys,kons boys kar read poem,poem kar read,is poem kar read by kons boys,John says kar kons boys read that poem,kons k

In [69]:
df.columns

Index(['sentence', 'subordinate-sentence', 'passive-sentence', 'it',
       'it-r-1-null_subject', 'it-r-2-passive', 'it-r-3-subordinate',
       'it-u-1-negation', 'it-u-2-invert', 'it-u-3-gender', 'jp-r-1-sov',
       'jp-r-2-passive', 'jp-r-3-subordinate', 'jp-u-1-negation',
       'jp-u-2-invert', 'jp-u-3-past-tense', 'ng-sentence',
       'ng-subordinate-sentence', 'ng-passive-sentence', 'ng-it',
       'ng-it-r-1-null_subject', 'ng-it-r-2-passive', 'ng-it-r-3-subordinate',
       'ng-it-u-1-negation', 'ng-it-u-2-invert', 'ng-it-u-3-gender',
       'ng-jp-r-1-sov', 'ng-jp-r-2-passive', 'ng-jp-r-3-subordinate',
       'ng-jp-u-1-negation', 'ng-jp-u-2-invert', 'ng-jp-u-3-past-tense'],
      dtype='object')

In [70]:
df.to_csv('ngs.csv')

## Generating Italian sentences

In [43]:
df

,sentence,subordinate-sentence,passive-sentence,it,it-r-1-null_subject,it-r-2-passive,it-r-3-subordinate,it-u-1-negation,it-u-2-invert,it-u-3-gender,...,ng-it-u-2-invert,ng-it-u-3-gender,ng-jp-r-1-sov,ng-jp-r-2-passive,ng-jp-r-3-subordinate,ng-jp-u-1-negation,ng-jp-u-2-invert,ng-jp-u-3-past-tense,ita_sentence,ita_subordinate-sentence
0,a dog carries the fish,Leela notices that the dog carries the fish,the fish is carried by a dog,kon dog carries kon fish,carries kon fish,kon fish is carried by kon dog,Leela notices that kon dog carries kon fish,kon dog carries kon no fish,fish kon carries dog kon,kon dog carries kon fish,...,fish kon carries kon dog,kon dog carries fish kon,a dog fish the wa o carries,the fish wa a dog reru to carry ni,to wa the dog ga the fish o carries Leela notices,a no wa dog the fish o carries,carries o fish wa the dog a,a dog the wa fish o-ta to carry,NaN,NaN
0,the dog holds a fish,Gomu says that the dog holds the fish,a fish is held by the dog,kon dog holds kon fish,holds kon fish,kon fish is held by kon dog,Gomu says that kon dog holds kon fish,kon dog holds kon no fish,fish kon holds dog kon,kon dog holds kon fish,...,kon fish holds dog kon,kon kon holds dog fish,the dog a wa fish o holds,a fish wa the dog to ni hold reru,Gomu o the dog ga the fish wa holds to says,the dog wa no o fish a holds,o holds fish a wa dog the,the dog wa a fish o-ta to hold,NaN,NaN
0,a dog takes a fish,Maria states that the dog takes the fish,a fish is taken by a dog,kon dog takes kon fish,takes kon fish,kon fish is taken by kon dog,Maria states that kon dog takes kon fish,kon dog takes kon no fish,fish kon takes dog kon,kon dog takes kon fish,...,kon kon takes dog fish,kon dog fish kon takes,a dog fish a wa o takes,a fish wa a dog ni reru take to,Maria wa fish dog ga the the o takes to states,a dog wa o a fish no takes,fish o takes a wa dog a,a dog wa fish a o-ta to take,NaN,NaN
0,the dog brings a fish,John sees that the dog brings the fish,a fish is brought by the dog,kon dog brings kon fish,brings kon fish,kon fish is brought by kon dog,John sees that kon dog brings kon fish,kon dog brings kon no fish,fish kon brings dog kon,kon dog brings kon fish,...,kon kon brings dog fish,kon brings dog kon fish,o dog wa a fish the brings,a fish wa the dog bring to ni reru,John brings the dog ga the fish o wa to sees,the wa dog no a fish o brings,brings fish o a wa dog the,the dog o-ta a fish wa to bring,NaN,NaN
0,the dog carries a mouse,Maria states that the dog carries the mouse,a mouse is carried by the dog,kon dog carries kon mouse,carries kon mouse,kon mouse is carried by kon dog,Maria states that kon dog carries kon mouse,kon dog carries kon no mouse,mouse kon carries dog kon,kon dog carries kon mouse,...,carries kon mouse dog kon,carries dog kon kon mouse,the dog wa carries mouse o a,a ni wa the dog mouse to carry reru,the wa Maria dog ga the mouse o carries to states,the no wa dog a mouse o carries,o carries mouse a wa dog the,the dog a wa mouse o-ta to carry,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,un' autore legge il capitolo,Gomu vede che un' autore legge il capitolo
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lo scrittore legge il libro,John osserva che lo scrittore legge il libro
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uno scrittore legge un poema,Leela dice che uno scrittore legge un poema
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uno scrittore legge una storia,Gomu vede che uno scrittore legge una storia


## IT Real grammar 1 (Null Subject parameter)

In [44]:


import random
for obj in it_seed:
    for subj in obj['subject']:
        for ob in obj['object']:
            for verb in obj['verb']:
                

In [46]:
df[["sentence", "subordinate-sentence", "passive-sentence", "ita_sentence", "ita_subordinate-sentence"]]

,sentence,subordinate-sentence,passive-sentence,ita_sentence,ita_subordinate-sentence
0,a dog carries the fish,Leela notices that the dog carries the fish,the fish is carried by a dog,NaN,NaN
0,the dog holds a fish,Gomu says that the dog holds the fish,a fish is held by the dog,NaN,NaN
0,a dog takes a fish,Maria states that the dog takes the fish,a fish is taken by a dog,NaN,NaN
0,the dog brings a fish,John sees that the dog brings the fish,a fish is brought by the dog,NaN,NaN
0,the dog carries a mouse,Maria states that the dog carries the mouse,a mouse is carried by the dog,NaN,NaN
...,...,...,...,...,...
0,NaN,NaN,NaN,una ragazza legge un capitolo,Harry afferma che una ragazza legge un capitolo
0,NaN,NaN,NaN,il ragazzo legge il libro,Gomu dice che il ragazzo legge il libro
0,NaN,NaN,NaN,il ragazzo legge un poema,Tom sa che il ragazzo legge un poema
0,NaN,NaN,NaN,il ragazzo legge la storia,John sa che il ragazzo legge la storia


In [ ]:
df['it'] = df.progress_apply(lambda row: " ".join([genderDict[row['sentence'].split(" ")[1]]] + row['sentence'].split(" ")[1:3] + [genderDict[row['sentence'].split(" ")[4]]] + [row['sentence'].split(" ")[-1]]), axis=1)

  0%|          | 0/812 [00:00<?, ?it/s]

## IT Real grammar 1 (Null Subject parameter)

In [ ]:
df['it-r-1-null_subject'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[2:]), axis=1)

  0%|          | 0/812 [00:00<?, ?it/s]

## IT Real Grammar 2 (Passive construction)

In [ ]:
df['it-r-2-passive'] =  df.progress_apply(lambda row: " ".join([genderDict[row['passive-sentence'].split(" ")[1]]] + row['passive-sentence'].split(" ")[1:-2] + [genderDict[row['passive-sentence'].split(" ")[-1]]] + [row['passive-sentence'].split(" ")[-1]]), axis=1)

  0%|          | 0/812 [00:00<?, ?it/s]

## IT Real Grammar 3 (Subordinate construction)

In [ ]:
df['it-r-3-subordinate'] =  df.progress_apply(lambda row: " ".join(row['subordinate-sentence'].split(" ")[0:3] + row['it'].split(" ")), axis=1)

  0%|          | 0/812 [00:00<?, ?it/s]

In [ ]:
df

,sentence,subordinate-sentence,passive-sentence,it,it-r-1-null_subject,it-r-2-passive,it-r-3-subordinate
0,a dog carries a fish,Tom sees that the dog carries the fish,a fish is carried by a dog,kon dog carries kon fish,carries kon fish,kon fish is carried by kon dog,Tom sees that kon dog carries kon fish
0,the dog holds the fish,Leela claims that the dog holds the fish,the fish is held by the dog,kon dog holds kon fish,holds kon fish,kon fish is held by kon dog,Leela claims that kon dog holds kon fish
0,a dog takes a fish,Tom sees that the dog takes the fish,a fish is taken by a dog,kon dog takes kon fish,takes kon fish,kon fish is taken by kon dog,Tom sees that kon dog takes kon fish
0,the dog brings the fish,John says that the dog brings the fish,the fish is brought by the dog,kon dog brings kon fish,brings kon fish,kon fish is brought by kon dog,John says that kon dog brings kon fish
0,a dog carries the mouse,Tom sees that the dog carries the mouse,the mouse is carried by a dog,kon dog carries kon mouse,carries kon mouse,kon mouse is carried by kon dog,Tom sees that kon dog carries kon mouse
...,...,...,...,...,...,...,...
0,the girls read the chapter,John notices that the girls read the chapter,the chapter is read by the girls,kars girls read kon chapter,read kon chapter,kon chapter is read by kars girls,John notices that kars girls read kon chapter
0,the boys read a book,Tom states that the boys read the book,a book is read by the boys,kons boys read kon book,read kon book,kon book is read by kons boys,Tom states that kons boys read kon book
0,the boys read a poem,Leela notices that the boys read the poem,a poem is read by the boys,kons boys read kar poem,read kar poem,kar poem is read by kons boys,Leela notices that kons boys read kar poem
0,the boys read a story,John states that the boys read the story,a story is read by the boys,kons boys read kar story,read kar story,kar story is read by kons boys,John states that kons boys read kar story


## IT Unreal Grammar 1: Add a negation after the 3rd word in the nullified subject sentence

In [ ]:
df['it-u-1-negation'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[:4] + [ "no" ] + row['sentence'].split(" ")[4:]), axis=1)

  0%|          | 0/812 [00:00<?, ?it/s]

## IT Unreal Grammar 2: Invert italian sentence

In [ ]:
df['it-u-2-invert'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[::-1]), axis=1)

  0%|          | 0/812 [00:00<?, ?it/s]

## IT Unreal Grammar 3: Same gender for subject and object

In [ ]:
df['it-u-3-gender'] = df.progress_apply(lambda row: " ".join(row['it'].split(" ")[:3] + [row['it'].split(" ")[0]] + [row['it'].split(" ")[-1]]), axis=1)

  0%|          | 0/812 [00:00<?, ?it/s]